# 1NN vs. RandomForest

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [50]:
from sklearn import datasets, metrics, ensemble
import pandas as pd
import numpy as np

In [35]:
def write_answer(num, answer):
    with open("week5_1nn_answer{}.txt".format(num), "w") as fout:
        fout.write(str(answer))

In [2]:
digits = datasets.load_digits()

In [6]:
X = pd.DataFrame(digits.data)
y = pd.DataFrame(digits.target).rename(columns={0 : 'target'})
pd.concat([X, y], axis=1).head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [70]:
trainsize = int(0.75 * digits.data.shape[0])
testsize = digits.data.shape[0] - trainsize
X_train = digits.data[:trainsize]  
y_train = digits.target[:trainsize]
X_test = digits.data[trainsize:]
y_test = digits.target[trainsize:]

# TASK 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeigborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [75]:
def one_nn(train_samples, train_labels, test_samples):
    min_distance = None
    cur_distance = None
    nearest = None
    nearest_label = None
    test_labels = []
    for test_sample in test_samples:
        min_distance = np.inf
        for train_sample, train_label in zip(train_samples, train_labels):
            cur_distance = metrics.pairwise.euclidean_distances(train_sample.reshape(1, -1), test_sample.reshape(1, -1))
            if (cur_distance <= min_distance):
                if train_label > nearest_label:
                    continue
                min_distance = cur_distance
                nearest = train_sample
                nearest_label = train_label
        test_labels.append(nearest_label)
    return test_labels

In [66]:
y_pred = one_nn(X_train, y_train, X_test)

In [84]:
score = metrics.accuracy_score(y_test, y_pred)
print("Accuracy score for 1NN : {}".format(1 - score))
write_answer(1, 1 - score)

Accuracy score for 1NN : 0.0377777777777778


# TASK 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [80]:
clf = ensemble.RandomForestClassifier(n_estimators=1000)
clf = clf.fit(X_train, y_train)

In [81]:
score = metrics.accuracy_score(y_test, clf.predict(X_test))
print("Accuracy score for RandomForestClassifier : {}".format(1 - score))
write_answer(2, score)

Accuracy score for RandomForestClassifier : 0.06444444444444442
